# Stock Strategy
### Based on 5 years of data obtained from yahoo finance

#### Sources

- https://github.com/kunalkini15/technical_indicators_lib
- https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py
- https://www.investopedia.com/

### To do

- remove old functions
- create a new function that takes an OHLC df and modifies it with all the indicators

_____

## Import modules

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
import seaborn as sns

In [2]:
from technical_indicators_lib.indicators import (
    ADI, ATR, BollingerBands, CCI, CMF, CHO, CHV, DC, DPO, EMV, EMA, FI, KC, KST, MI, MED, MOM,
    MFI, MACD, NVI, OBV, PVT, PC, PO, ROC, ROCV, RSI, SMA, VLT, StochasticKAndD,
    Trix, TR, TSI, TYP, VHF, VO, WCL)

adi = ADI()
atr = ATR()
bb = BollingerBands()
cci = CCI()
cmf = CMF()
cho = CHO()
chv = CHV()
dc = DC()
dpo = DPO()
emv = EMV()
ema = EMA()
fi = FI()
kc = KC()
kst = KST()
mi = MI()
mem = MED()
mom = MOM()
mfi = MFI()
macd = MACD()
nvi = NVI()
obv = OBV()
pvt = PVT()
pc = PC()
po = PO()
roc = ROC()
rocv = ROCV()
rsi = RSI()
sma = SMA()
vlt = VLT()
stochastic_k_and_d = StochasticKAndD()
trix = Trix()
tr = TR()
tsi = TSI()
typ = TYP()
vhf = VHF()
vo = VO()
wcl = WCL()

## Get data from YahooFinance API

In [3]:
def get_yf_data(ticker, time_period='5y'):
    '''
    Calls Yahoo Finance API and retrieves historical data for given ticker and time period
    
    Args:
        ticker(string): string, stock ticker
        time: length of time for data (default: 5 years)
        
    Returns:
        pandas.DataFrame: new pandas dataframe with the OHLC data for the given ticker and time period
    '''
    # get historical data
    temp = yf.Ticker(ticker)
    df = temp.history(period=time_period)
    # make columns lowercase (compatibility with technical_indicators_lib)
    df.columns = map(str.lower, df.columns)
    
    return df   

___

## Add Technical Indicators to df


#### Moving Average Function

In [4]:
# moving average
def moving_average(df, n):
    '''Calculate the moving average of closing price over n days'''
    name = 'MA_' + str(n) + 'd'
    df[name] = df['close'].rolling(window=n, min_periods=n).mean()
    return df

#### Calculate Indicators for each df

In [12]:
def get_technical_indicators(df):
    '''
    Adds selected technical indicators as columns to the dataframe
    
    Args:
        df(pandas.DataFrame): OHLC dataframe retrieved from yahoofinanace api
        
    Returns:
        pandas.DataFrame: modified pandas dataframe with select technical indicators
    '''
    
    # suppress pandas copy warning
    df.is_copy = False
    
    # calculate select moving averages
    d = [5, 8, 10, 13, 20, 21, 34, 50, 55, 89, 100]
    for day in d:
        moving_average(df, day)
        
    # calculate exponential moving averages
    d_ema = [5, 12, 20, 26, 50, 200]
    for day in d_ema:
        df = ema.get_value_df(df, time_period=day)
        col_name = 'EMA_' + str(day) + 'd'
        df.rename(columns = {'EMA': col_name}, inplace=True)
        
    # calculate on-balance volume
    df = obv.get_value_df(df)
    
    # calculate accumulation/distribution
    df = adi.get_value_df(df)
    
    # calculate moving average convergence
    df = macd.get_value_df(df)
    
    # calculate relative strength index
    df = rsi.get_value_df(df)
    
    # calculate stochastic oscillator (k and d)
    df = stochastic_k_and_d(df)
    
    # calculate bollinger bands
    df = bb.get_value_df(df)
    
    # calculate price volume trend
    df = pvt.get_value_df(df)
    
    # calculate average true range
    df = atr.get_value_df(df)
    
    # calculate commodity channel index
    df = cci.get_value_df(df)
    
    # calculate chaikin money flow
    df = cmf.get_value_df(df)
    
    # calculate chaikin oscillators
    df = cho.get_value_df(df)
    
    # calculate chaikin volatility
    df = chv.get_value_df(df)
    
    # calculate donchian channel
    df = dc.get_value_df(df)
    
    # calculate detrended price oscillator
    df = dpo.get_value_df(df)
    
    # calculate ease of movement

    return df

In [17]:
print(rocv.info())

ROCV indicator is used to identify whether the price movement is confirmed by trading volume.

 Links:
http://www.ta-guru.com/Book/TechnicalAnalysis/TechnicalIndicators/VolumeRateOfChange.php5
https://www.investopedia.com/articles/technical/02/091002.asp



In [10]:
QQQ = get_yf_data('QQQ')

In [11]:
QQQ = get_technical_indicators(QQQ)

In [13]:
QQQ.head(50)

,open,high,low,close,volume,dividends,stock splits,MA_5d,MA_8d,MA_10d,...,EMA_12d,EMA_20d,EMA_26d,EMA_50d,EMA_200d,OBV,ADI,MACD,MACD_signal_line,RSI
Date,,,,,,,,,,,,,,,,,,,,,
2016-03-28,103.265090,103.284295,102.650646,102.833061,13892300,0.0,0,NaN,NaN,NaN,...,102.833061,102.833061,102.833061,102.833061,102.833061,13892300.0,-1.178729e+07,0.000000,0.000000,NaN
2016-03-29,102.641051,104.551587,102.468238,104.484383,29963900,0.0,0,NaN,NaN,NaN,...,103.727527,103.700005,103.690478,103.675235,103.662850,43856200.0,1.624346e+07,0.037049,0.020583,NaN
2016-03-30,105.108444,105.646080,104.772413,104.993233,23807200,0.0,0,NaN,NaN,NaN,...,104.221532,104.174870,104.158545,104.132256,104.110753,67663400.0,4.470799e+06,0.062987,0.037962,NaN
2016-03-31,104.916407,105.310039,104.676390,104.839600,21447200,0.0,0,NaN,NaN,NaN,...,104.416632,104.366767,104.348938,104.319840,104.295707,46216200.0,-5.928024e+06,0.067694,0.048033,NaN
2016-04-01,104.205966,105.991697,104.061954,105.953293,26389000,0.0,0,104.620714,NaN,NaN,...,104.834137,104.750534,104.720995,104.673178,104.633887,72605200.0,1.941064e+07,0.113143,0.067402,NaN
2016-04-04,105.953292,106.030099,105.300444,105.550064,17163000,0.0,0,105.164114,NaN,NaN,...,105.008145,104.919199,104.887051,104.834322,104.790426,55442200.0,1.399081e+07,0.121094,0.081956,NaN
2016-04-05,104.733993,105.127625,104.397970,104.532379,20107900,0.0,0,105.173714,NaN,NaN,...,104.901980,104.846061,104.823974,104.785842,104.752447,35334300.0,1.291034e+06,0.078005,0.080956,NaN
2016-04-06,104.637983,106.289304,104.637983,106.250900,25056000,0.0,0,105.425247,104.929614,NaN,...,105.183479,105.088894,105.053887,104.995616,104.946375,60390300.0,2.518161e+07,0.129591,0.092644,NaN
2016-04-07,105.636474,105.828493,104.273177,104.724411,28567200,0.0,0,105.402209,105.166033,NaN,...,105.092659,105.030429,105.005052,104.960441,104.920714,31823100.0,1.319046e+07,0.087607,0.091480,NaN


### Candle Charts

In [ ]:
def candle_plot(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'])])
    fig.show()

In [ ]:
candle_plot(QQQ)